# Interconnecting an OVS Network with an LLM via FabNet 

This notebook demonstrates how to set up and connect an Open vSwitch (OVS) network across multiple FABRIC sites, enabling communication with an LLM running in a different slice over FabNet.  

### Background: OVS  

Open vSwitch ([OVS](https://www.openvswitch.org/)) is an open-source, multi-layer virtual switch designed for virtualized environments. It plays a key role in many software-defined networking (SDN) and virtualization platforms.  

### Target FABRIC Topology  

In this setup, three OVS bridges are deployed across three different FABRIC sites, forming a ring topology. Each bridge is connected to a VM at the same site, and both the bridge and the VM are linked to the FABNetv4 network.  

We then demonstrate traffic flow between the three VMs and run network tests—including `ping`, `iperf`, `traceroute`, and `mtr`. The test results are forwarded to an LLM running on a separate FABRIC slice connected to FabNetv4, and the LLM generates responses based on the analysis.  

A high-level view of the topology is illustrated below.  

<img src="./figs/openvswitch.png" width="70%"><br>  

### Host Placement Considerations  

Due to NVIDIA/Mellanox constraints, when using `NIC_Basic` for an OVS bridge experiment, it is recommended to deploy the bridge VM on a separate host from the VMs connected to the bridge.  

However, this restriction does not apply to `NIC_ConnectX_5` and `NIC_ConnectX_6` configurations.

## Import the FABlib Library

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network
import ipaddress
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager


fablib = fablib_manager()
fablib.show_config();

## Create the Experiment Slice

### General Configuration

Set the name of the slice, the sites to use, and the number of additional (non-router) nodes to add to each site.

In [ ]:
slice_name= "MySlice-openvswitch"

host_column_name = "hosts"
ram_column_name = "ram_available"
disk_column_name = "disk_available"
core_column_name = "cores_available"
sites = [site1, site2, site3] = fablib.get_random_sites(count=3, 
                                                        filter_function=lambda x: x[host_column_name] > 1 and 
                                                        x[core_column_name] > 8 and 
                                                        x[ram_column_name] > 16 and
                                                        x[disk_column_name] > 100, 
                                                        avoid=["NEWY", "BRIST", "MICH", "HAWI", "RUTG"])
print(f"Sites: {sites}")

site_node_count = 3
bridge_name_prefix = "bridge"

### Create Slice

In [ ]:
slice = fablib.new_slice(name=slice_name)
default_image= 'default_ubuntu_22'

In [ ]:
from ipaddress import IPv4Network

l2_subnet = IPv4Network("10.10.10.0/24")
available_ips = list(l2_subnet)[1:]

### Add Bridge1

Because of constraints imposed by NVIDIA/Mellanox, when utilizing `NIC_Basic` for an OVS bridge experiment, it is advisable to deploy the VM responsible for running the bridge on a separate host from the VMs linked to the bridge.

`bridge1` in the example below is requested on host `{site1.lower()}-w1.fabric-testbed.net`

In [ ]:
bridge1 = slice.add_node(name=f"{bridge_name_prefix}-1", site=site1, cores=4, ram=8, disk=50, image=default_image, host=f"{site1.lower()}-w1.fabric-testbed.net")
bridge1_nic1 = bridge1.add_component(model='NIC_Basic', name='nic_local_1')
bridge1_nic2 = bridge1.add_component(model='NIC_Basic', name='nic_local_2')
bridge1_nic3 = bridge1.add_component(model='NIC_Basic', name='nic_local_3')
bridge1.add_post_boot_execute('yes | sudo apt-get -y update && sudo apt-get upgrade -y') 
bridge1.add_post_boot_execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common')
bridge1.add_post_boot_execute('sudo apt-get -y install net-tools traceroute')


## Add Bridge2
Because of constraints imposed by NVIDIA/Mellanox, when utilizing `NIC_Basic` for an OVS bridge experiment, it is advisable to deploy the VM responsible for running the bridge on a separate host from the VMs linked to the bridge.

`bridge2` in the example below is requested on host `{site2.lower()}-w1.fabric-testbed.net`

In [ ]:
bridge2 = slice.add_node(name=f"{bridge_name_prefix}-2", site=site2, cores=4, ram=8, disk=50, image=default_image, host=f"{site2.lower()}-w1.fabric-testbed.net")
bridge2_nic1 = bridge2.add_component(model='NIC_Basic', name='nic_local_1')
bridge2_nic2 = bridge2.add_component(model='NIC_Basic', name='nic_local_2')
bridge2_nic3 = bridge2.add_component(model='NIC_Basic', name='nic_local_3')
bridge2.add_post_boot_execute('yes | sudo apt-get -y update && sudo apt-get upgrade -y') 
bridge2.add_post_boot_execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common')
bridge2.add_post_boot_execute('sudo apt-get -y install net-tools traceroute')

### Add Bridge3
Because of constraints imposed by NVIDIA/Mellanox, when utilizing `NIC_Basic` for an OVS bridge experiment, it is advisable to deploy the VM responsible for running the bridge on a separate host from the VMs linked to the bridge.

`bridge3` in the example below is requested on host `{site3.lower()}-w1.fabric-testbed.net`

In [ ]:
bridge3 = slice.add_node(name=f"{bridge_name_prefix}-3", site=site3, cores=4, ram=8, disk=50, image=default_image, host=f"{site3.lower()}-w1.fabric-testbed.net")
bridge3_nic1 = bridge3.add_component(model='NIC_Basic', name='nic_local_1')
bridge3_nic2 = bridge3.add_component(model='NIC_Basic', name='nic_local_2')
bridge3_nic3 = bridge3.add_component(model='NIC_Basic', name='nic_local_3')
bridge3.add_post_boot_execute('yes | sudo apt-get -y update && sudo apt-get upgrade -y') 
bridge3.add_post_boot_execute('yes | sudo apt-get -y install openvswitch-switch openvswitch-common')
bridge3.add_post_boot_execute('sudo apt-get -y install net-tools traceroute')

### Add Host Nodes

Host nodes are required to be located on a host other than `bridge1`, `bridge2` and `bridge4`, specifically on `{site{x}.lower()}-w2.fabric-testbed.net`. These nodes can be distributed across any other host, or all hosted on the same host, provided it is not the `bridge1`, `bridge2` or `bridge3` host.

In [ ]:
for node_num in range(1, site_node_count + 1):
    site = sites[node_num -1]
    idx = node_num-1
    
    print(f"Adding nodes to {site}")
    node_name = f"{site.lower()}"
    bridge_name = f"{bridge_name_prefix}-{node_num}"

    net1 = slice.add_l2network(name=f"{node_name}_l2", subnet=l2_subnet)

    node = slice.add_node(name=node_name, site=site, cores=4, ram=8, disk=50, image=default_image, host=f"{site.lower()}-w2.fabric-testbed.net")
    iface1 = node.add_component(model='NIC_Basic', name='nic_l2').get_interfaces()[0]    
    iface1.set_mode("config")  
    net1.add_interface(iface1)
    iface1.set_ip_addr(available_ips[idx])
    
    bridge_node = slice.get_node(name=bridge_name)
    bridge_ifc = bridge_node.get_component(name='nic_local_1').get_interfaces()[0]
    bridge_ifc.set_mode("config")  
    net1.add_interface(bridge_ifc)
    bridge_ifc.set_ip_addr(available_ips[idx + 1])


    iface2 = node.add_component(model='NIC_Basic', name='nic_l3').get_interfaces()[0]    
    iface2.set_mode("auto")
    net2 = slice.add_l3network(name=f"{node_name}_l3")
    net2.add_interface(iface2)

    bridge_nic_l3_ifc = bridge_node.add_component(model='NIC_Basic', name='nic_l3').get_interfaces()[0]
    bridge_nic_l3_ifc.set_mode("auto")
    net2.add_interface(bridge_nic_l3_ifc)


    node.add_post_boot_execute("sudo apt-get -y install net-tools traceroute")
    node.add_post_boot_execute("curl https://downloads.es.net/pub/iperf/iperf-3.13-mt1.tar.gz > iperf-3.13-mt1.tar.gz")
    node.add_post_boot_execute("tar -zxvf iperf-3.13-mt1.tar.gz")
    node.add_post_boot_execute("sudo DEBIAN_FRONTEND=noninteractive apt update -q -y")
    node.add_post_boot_execute("sudo DEBIAN_FRONTEND=noninteractive apt install -q -y build-essential")
    node.add_post_boot_execute("cd iperf-3.13-mt1/; sudo DEBIAN_FRONTEND=noninteractive  ./configure; sudo DEBIAN_FRONTEND=noninteractive make; sudo DEBIAN_FRONTEND=noninteractive make install")
    node.add_post_boot_execute("echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib' >> ~/.bashrc")
    

### Connect the Bridges

In [ ]:
net_br1_br2 = slice.add_l2network(name="br1_br2")
net_br1_br2.add_interface(bridge1_nic2.get_interfaces()[0])
net_br1_br2.add_interface(bridge2_nic2.get_interfaces()[0])

net_br1_br3 = slice.add_l2network(name="br1_br3")
net_br1_br3.add_interface(bridge1_nic3.get_interfaces()[0])
net_br1_br3.add_interface(bridge3_nic2.get_interfaces()[0])

net_br2_br3 = slice.add_l2network(name="br2_br3")
net_br2_br3.add_interface(bridge2_nic3.get_interfaces()[0])
net_br2_br3.add_interface(bridge3_nic3.get_interfaces()[0])

### Submit the Slice Request

<img src="./figs/openvswitch-single.png" width="60%"><br>

Now that the slice request is complete, you can submit it.


In [ ]:
slice.submit();

### Create a new bridge, enable the spanning tree protocol on necessary interfaces

In [ ]:
try:
    for node in slice.get_nodes():
        if node.get_name().startswith("bridge"):
            stdout, stderr = node.execute('sudo ovs-vsctl add-br br0')
            for interface in node.get_interfaces():
                stdout, stderr = node.execute(f'sudo ovs-vsctl add-port br0 {interface.get_physical_os_interface_name()}')
                #Remove IP addresses for all interfaces
                stdout, stderr = node.execute(f'sudo ifconfig {interface.get_physical_os_interface_name()} 0')
    
            #bring the bridge up
            stdout, stderr = node.execute('sudo ifconfig br0 up')
    print("Done")
except Exception as e:
    print(f"Exception: {e}")

### Enable Spanning tree and confirm

In [ ]:
for node in slice.get_nodes():
    if node.get_name().startswith("bridge"):
        stdout, stderr = node.execute('sudo ovs-vsctl set bridge br0 stp_enable=true')

In [ ]:
for node in slice.get_nodes():
    if node.get_name().startswith("bridge"):
        stdout, stderr = node.execute('sudo ovs-appctl stp/show')
        print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
        print()

## Verify connectivity between host nodes

In [ ]:
host1 = slice.get_node(name=f'{site1.lower()}')
host2 = slice.get_node(name=f'{site2.lower()}')
host3 = slice.get_node(name=f'{site3.lower()}')

In [ ]:
# Ping test
host2_ip_addr = host2.get_interface(network_name=f"{host2.get_name()}_l2").get_ip_addr()
host3_ip_addr = host3.get_interface(network_name=f"{host3.get_name()}_l2").get_ip_addr()
stdout, stderr = host1.execute(f'ping {host2_ip_addr} -c 5')

# Ping test
stdout, stderr = host1.execute(f'ping {host3_ip_addr} -c 5')

## Enable Access in all nodes to access other Notes Across FABRIC Internet(FabNet)

Configure all the nodes in the slice connected to FabNetv4 to be accessible from any VM running across FABRIC on FabNetV4 by setting up the necessary routes.


In [ ]:
slice = fablib.get_slice(slice_name)
for n in slice.get_nodes():
    network_name = f"{n.get_site().lower()}_l3"
    fabnet_network = slice.get_network(network_name)

    n.add_route(subnet=fablib.FABNETV4_SUBNET, next_hop=fabnet_network.get_gateway())
    n.config_routes()

    stdout, stderr = n.execute("sudo ip route list")
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
    print()

## Ollama Node Configuration in a Separate Slice

Please update the **IP address, model in use, and port number** for the Ollama node operating in a different slice and connected to **FabNetV4**. This LLM will be utilized to analyze test results from hosts connected to the **OVS Bridges** in the current slice.

In [ ]:
ollama_fabnet_ip_addr = "10.139.129.2"
ollama_model = "deepseek-r1:7b"
ollama_port = "11434"

## Execute Tests

We will now conduct a series of tests between `host1`, `host2`, and `host3`, which are interconnected through a ring of **OVS switches**. The test results will be sent to the **LLM**, running in a different slice, for analysis.

### Upload test scripts

In [ ]:
host1.upload_directory("./tools", ".")

### Execute Ping Tests

Perform **ping tests** from `host1` to both `host2` and `host3`. Forward the output to the **LLM** for comparative analysis.

In [ ]:
print(f"Running ping test SRC: {host1.get_name()} DEST: {host2.get_name()}, {host3.get_name()}")

stdout, stderr = host1.execute(f'python3 tools/net_llm_tester.py --test_type ping --dest_ips {host2_ip_addr} {host3_ip_addr} --ollama_model {ollama_model} --ollama_host {ollama_fabnet_ip_addr}  --ollama_port {ollama_port}')

### Execute MTR Tests  

Run **MTR (My Traceroute) tests** from `host1` to both `host2` and `host3`. Send the output to the **LLM** for comparative analysis.

In [ ]:
print(f"Running mtr test SRC: {host1.get_name()} DEST: {host2.get_name()}, {host3.get_name()}")

stdout, stderr = host1.execute(f'python3 tools/net_llm_tester.py --test_type mtr --dest_ips {host2_ip_addr} {host3_ip_addr} --ollama_model {ollama_model} --ollama_host {ollama_fabnet_ip_addr}  --ollama_port {ollama_port}')

### Execute Traceroute Tests  

Run **traceroute** from `host1` to both `host2` and `host3`. Forward the results to the **LLM** for comparative analysis.

In [ ]:
print(f"Running traceroute test SRC: {host1.get_name()} DEST: {host2.get_name()}, {host3.get_name()}")

stdout, stderr = host1.execute(f'python3 tools/net_llm_tester.py --test_type traceroute --dest_ips {host2_ip_addr} {host3_ip_addr} --ollama_model {ollama_model} --ollama_host {ollama_fabnet_ip_addr}  --ollama_port {ollama_port}')

### Execute iPerf3 Tests  

Start **iPerf3** in **server mode** on `host2` and in **client mode** on `host1`. Send the output to the **LLM** for analysis.

In [ ]:
print(f"Starting iperf3 in server mode on {host2.get_name()}")
host2.execute_thread("export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/local/lib && iperf3 -s -1")

In [ ]:
print(f"Running iperf3 test Client: {host1.get_name()} Server: {host2.get_name()}")
stdout, stderr = host1.execute(f'python3 tools/net_llm_tester.py --test_type iperf --dest_ips {host2_ip_addr} --ollama_model {ollama_model} --ollama_host {ollama_fabnet_ip_addr}  --ollama_port {ollama_port}')

### Delete the slice

In [ ]:
slice.delete()